In [20]:
from shutil import rmtree

from fmpy import *
from fmpy.fmi3 import *

fmu_filename = 'BouncingBall.fmu'
csv_filename = 'BouncingBall_out.csv'

unzipdir = extract(filename)

model_description = read_model_description(unzipdir)

with open(csv_filename, 'w') as f:

    f.write('time,h,v\n')

    vr = dict((v.name, v.valueReference) for v in model_description.modelVariables)

    def intermediate_update(instanceEnvironment,
                            intermediateUpdateTime,
                            intermediateVariableSetRequested,
                            intermediateVariableGetAllowed,
                            intermediateStepFinished,
                            canReturnEarly,
                            earlyReturnRequested,
                            earlyReturnTime):

        global fmu_instance

        h, v = fmu_instance.getFloat64([vr['h'], vr['v']])

        f.write(f'{intermediateUpdateTime},{h},{v}\n')

        earlyReturnRequested.contents = fmi3False


    fmu_instance = instantiate_fmu(unzipdir=unzipdir, model_description=model_description, intermediate_update=intermediate_update)

    result = simulate_fmu(filename, model_description=model_description, fmu_instance=fmu_instance,
                          output_interval=0.01)

    fmu_instance.freeInstance()

rmtree(unzipdir)

d = read_csv(csv_filename)

fig = go.Figure()

time = result['time']

fig.add_trace(
    go.Scatter(
        name='communication points',
        mode='markers',
        x=time,
        y=result['h'],
        marker=dict(color='#636efa', size=15)
    )
)

fig.add_trace(
    go.Scatter(
        name='internal steps',
        mode='lines+markers',
        x=d['time'],
        y=d['h'],
        marker=dict(color='#636efa', size=5)
    )
)

axes_attrs = dict(showgrid=True, gridwidth=1, ticklen=0, gridcolor='LightGrey', linecolor='black', showline=True, zerolinewidth=1, zerolinecolor='LightGrey')
fig.update_xaxes(range=(0.745, 0.795), **axes_attrs) 
fig.update_yaxes(range=(0.491, 0.495), **axes_attrs)
fig.update_layout(margin=dict(t=30, b=0, r=30), plot_bgcolor='rgba(0,0,0,0)')

fig.show()
